In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [2]:
# Set your dataset path
dataset_path = '/content/drive/MyDrive/dataset'

In [ ]:
# Load CSV files
train_data = pd.read_csv(os.path.join(dataset_path, 'train.csv'))

# Add .jpg extension to filenames
train_data['image_id'] = train_data['image_id'].apply(lambda x: x + '.jpg')

In [ ]:
# Split train_data into training and validation sets
train_df, val_df = train_test_split(train_data, test_size=0.2, random_state=42)

# Set paths for image folders
train_images_path = os.path.join(dataset_path, 'train_images')

In [ ]:
# Define transformations for training and validation sets (Resizing to 128x128)
train_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(40),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Custom dataset class
class PlantDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.dataframe.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        label = torch.tensor(self.dataframe.iloc[idx, 1:].values.astype('float32'))

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
# Create DataLoader objects
train_dataset = PlantDataset(train_df, train_images_path, transform=train_transforms)
val_dataset = PlantDataset(val_df, train_images_path, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [ ]:
# Define the model using ResNet-18 (smaller, faster model)
class PlantDiseaseModel(nn.Module):
    def __init__(self):
        super(PlantDiseaseModel, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, 4)

    def forward(self, x):
        return self.model(x)

model = PlantDiseaseModel()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 70.1MB/s]


PlantDiseaseModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, t

In [ ]:
# Mixed precision training for faster computation
from torch.cuda.amp import autocast, GradScaler
scaler = GradScaler()

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

<ipython-input-9-b7d01d6dd22a>:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/usr/local/lib/python3.10/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
# Early stopping criteria
patience = 3
early_stop_counter = 0
best_accuracy = 0.0

In [ ]:
# Training and evaluation loop with mixed precision
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=15):
    global best_accuracy, early_stop_counter

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)

                with autocast():
                    outputs = model(images)
                    loss = criterion(outputs, labels)

                val_loss += loss.item()

                # Convert logits to probabilities and then to binary labels
                preds = torch.sigmoid(outputs)
                preds = preds > 0.5
                correct += (preds == labels).sum().item()
                total += labels.size(0) * labels.size(1)

        scheduler.step(val_loss / len(val_loader))

        val_accuracy = correct / total
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            early_stop_counter = 0
            torch.save(model.state_dict(), '/content/drive/MyDrive/best_plant_disease_model.pth')
        else:
            early_stop_counter += 1

        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {running_loss/len(train_loader):.4f}, "
              f"Val Loss: {val_loss/len(val_loader):.4f}, "
              f"Val Accuracy: {val_accuracy:.4f}, "
              f"Best Accuracy: {best_accuracy:.4f}")

        # Early stopping
        if early_stop_counter >= patience:
            print("Early stopping...")
            break

In [ ]:
# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=15)

<ipython-input-11-8c24fb065eca>:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:265: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
<ipython-input-11-8c24fb065eca>:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch [1/15], Train Loss: 0.4627, Val Loss: 0.3128, Val Accuracy: 0.9027, Best Accuracy: 0.9027
Epoch [2/15], Train Loss: 0.2348, Val Loss: 0.1929, Val Accuracy: 0.9322, Best Accuracy: 0.9322
Epoch [3/15], Train Loss: 0.1632, Val Loss: 0.1710, Val Accuracy: 0.9377, Best Accuracy: 0.9377
Epoch [4/15], Train Loss: 0.1325, Val Loss: 0.1667, Val Accuracy: 0.9411, Best Accuracy: 0.9411
Epoch [5/15], Train Loss: 0.1167, Val Loss: 0.1502, Val Accuracy: 0.9514, Best Accuracy: 0.9514
Epoch [6/15], Train Loss: 0.0995, Val Loss: 0.1256, Val Accuracy: 0.9575, Best Accuracy: 0.9575
Epoch [7/15], Train Loss: 0.0985, Val Loss: 0.1253, Val Accuracy: 0.9562, Best Accuracy: 0.9575
Epoch [8/15], Train Loss: 0.1004, Val Loss: 0.1363, Val Accuracy: 0.9555, Best Accuracy: 0.9575
Epoch [9/15], Train Loss: 0.0725, Val Loss: 0.1299, Val Accuracy: 0.9596, Best Accuracy: 0.9596
Epoch [10/15], Train Loss: 0.0747, Val Loss: 0.1251, Val Accuracy: 0.9575, Best Accuracy: 0.9596
Epoch [11/15], Train Loss: 0.0572, Val 